## 01. Pemahaman Domain (Domain Understanding)

---



In [2]:
import pandas as pd
import re
import nltk
import ipykernel

In [54]:
data1 = pd.read_csv('crawling_tweet/timnas_1.csv')
data2 = pd.read_csv('crawling_tweet/timnas_2.csv')
data3 = pd.read_csv('crawling_tweet/timnas_3.csv')
data4 = pd.read_csv('crawling_tweet/timnas_4.csv')
data5 = pd.read_csv('crawling_tweet/timnas (6).csv')
data6 = pd.read_csv('crawling_tweet/timnas (7).csv')
data7 = pd.read_csv('crawling_tweet/timnas (8).csv')
data8 = pd.read_csv('crawling_tweet/timnas (9).csv')
data9 = pd.read_csv('crawling_tweet/timnas (10).csv')
data10 = pd.read_csv('crawling_tweet/timnas (11).csv')
data11 = pd.read_csv('crawling_tweet/timnas (12).csv')

In [55]:
data = pd.concat([data1, data5,data6,data7,data8,data9,data10,data11], ignore_index=True)

In [56]:
data 

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1800679330726715550,Tue Jun 11 23:59:42 +0000 2024,1,TIMNAS PRIA DAN WANITA MENANG + ERIK TEN HAG STAY,1800679330726715550,NaN,NaN,in,Indonesia,0,0,0,https://x.com/tigapuluhtigax/status/1800679330...,1188470356668895234,tigapuluhtigax
1,1800679305925800015,Tue Jun 11 23:59:36 +0000 2024,1,ngerasa ga sih sekarang timnas kalo main pakek...,1800679305925800015,NaN,NaN,in,"Jombang, Indonesia",0,0,0,https://x.com/sotongism/status/180067930592580...,961056397063409664,sotongism
2,1800679288326476196,Tue Jun 11 23:59:31 +0000 2024,0,2 Sejarah Dicetak Timnas Indonesia Usai Ganyan...,1800679288326476196,NaN,NaN,in,NaN,0,0,0,https://x.com/peanutkacang123/status/180067928...,1464596330421100548,peanutkacang123
3,1800589420766355730,Tue Jun 11 23:59:28 +0000 2024,0,@womensfootie_id Tinggal tmbah pemain diaspora...,1800679273185047018,NaN,womensfootie_id,in,NaN,0,0,0,https://x.com/PKananku1927/status/180067927318...,1772880651248939008,PKananku1927
4,1800679270857286043,Tue Jun 11 23:59:27 +0000 2024,0,Timnas Indonesia Vs Timnas Filipina 2-0 Indone...,1800679270857286043,NaN,NaN,in,NaN,0,0,0,https://x.com/kangsil2012/status/1800679270857...,546890263,kangsil2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3059,1800665012903813409,Tue Jun 11 23:02:48 +0000 2024,0,Daftar Negara Lolos ke Putaran 3 Kualifikasi P...,1800665012903813409,NaN,NaN,in,"Balikpapan, Indonesia",0,0,0,https://x.com/tribunkaltim/status/180066501290...,103551225,tribunkaltim
3060,1800580031087014330,Tue Jun 11 17:25:07 +0000 2024,1,Anang Ngerusak Momen Haru #TimNasDay Emang dar...,1800580031087014330,NaN,NaN,in,Salatiga - Jakarta - London,0,0,0,https://x.com/ronitoxid/status/180058003108701...,39716466,ronitoxid
3061,1800634437505343804,Tue Jun 11 21:01:18 +0000 2024,1,STY blak-blakan mengatakan stadion harusnya le...,1800634437505343804,NaN,NaN,in,Jakarta Capital Region,1,1,1,https://x.com/suaradotcom/status/1800634437505...,2340155395,suaradotcom
3062,1800672173994262918,Tue Jun 11 23:31:15 +0000 2024,0,Momen Timnas berlaga di Stadion jangan sekali-...,1800672173994262918,NaN,NaN,in,NaN,0,0,0,https://x.com/macanutara_/status/1800672173994...,1135805814642335745,macanutara_


In [57]:
data = data.drop_duplicates(subset='tweet_url')


In [59]:
jumlah_baris = data.shape[0]
print(f'Jumlah baris: {jumlah_baris}')

Jumlah baris: 1914


In [45]:
data.to_csv('crawling_tweet/timnas.csv', index=False)

## 1. Preposessing

In [46]:
import pandas as pd
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory


# Create stopword list
factory = StopWordRemoverFactory()
stop_words = set(factory.get_stop_words())

# Tambahkan kata hubung tambahan ke dalam daftar stopwords
kata_hubung = {'dan', 'atau', 'tetapi', 'namun', 'sehingga', 'karena', 'jika', 'jikalau', 'maka', 'sebab', 'meskipun', 'walaupun', 'yg','lu','gue','gini','klo','sama'}
stop_words.update(kata_hubung)

# Function to preprocess tweets
def preprocess_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)  # Remove URLs
    tweet = re.sub(r'@\w+', '', tweet)  # Remove mentions
    tweet = re.sub(r'#\w+', '', tweet)  # Remove hashtags
    tweet = re.sub(r'\d+', '', tweet)  # Remove numbers
    tweet = tweet.lower()  # Convert to lowercase
    tweet = re.sub(r'[^\w\s]', '', tweet)  # Remove punctuation
    tweet = tweet.split()  # Split into words
    tweet = [word for word in tweet if word not in stop_words]  # Remove stopwords
    return ' '.join(tweet)

# Apply preprocessing to each tweet
data['cleaned_tweet'] = data['full_text'].apply(preprocess_tweet)


C:\Users\TEMP\AppData\Local\Temp\ipykernel_6692\2347096102.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cleaned_tweet'] = data['full_text'].apply(preprocess_tweet)


## 2. Topic Modeling dengan LDA

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Vectorize the cleaned tweets
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words=list(stop_words))
tweet_matrix = vectorizer.fit_transform(data['cleaned_tweet'])

# Fit the LDA model
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(tweet_matrix)

# Display the top words for each topic
def display_topics(model, feature_names, num_top_words):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        topics.append([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]])
    return topics

num_top_words = 10
feature_names = vectorizer.get_feature_names_out()
topics = display_topics(lda, feature_names, num_top_words)

for idx, topic in enumerate(topics):
    print(f"Topik {idx+1}: {', '.join(topic)}")


c:\Users\TEMP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kurangnya', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


Topik 1: indonesia, timnas, stadion, filipina, gelora, karno, pot, vs, qatar, skor
Topik 2: timnas, kasih, terima, anak, min, kalo, you, back, ga, indonesia
Topik 3: timnas, nonton, ga, aja, indonesia, main, udah, bang, bgt, gak
Topik 4: timnas, anang, ernando, indonesia, bang, jay, syifa, filipina, paes, adrian
Topik 5: indonesia, timnas, piala, dunia, lolos, kualifikasi, asia, filipina, ketiga, putaran
Topik 6: timnas, anang, indonesia, lagu, nyanyi, pusaka, pemain, banget, suporter, tanah
Topik 7: timnas, indonesia, pemain, haye, thom, gol, laga, garuda, udah, bahrain
Topik 8: timnas, indonesia, main, gbk, gak, pemain, ridho, debut, nih, rizky
Topik 9: timnas, menang, indonesia, full, vs, kemenangan, bikin, nonton, senyum, liat
Topik 10: timnas, selamat, indonesia, kalo, juni, cup, hasil, amp, result, world


## 3. Clustering

In [48]:
import numpy as np

# Get topic distribution for each tweet
topic_distribution = lda.transform(tweet_matrix)

# Assign each tweet to the topic with the highest probability
data['topic_cluster'] = np.argmax(topic_distribution, axis=1)

# Save the DataFrame to a CSV file
data.to_csv('tweets_with_topics.csv', index=False)

# Display the first few rows with assigned topics
print(data[['full_text', 'topic_cluster']].head())


                                           full_text  topic_cluster
0  TIMNAS PRIA DAN WANITA MENANG + ERIK TEN HAG STAY              7
1  ngerasa ga sih sekarang timnas kalo main pakek...              2
2  2 Sejarah Dicetak Timnas Indonesia Usai Ganyan...              4
3  @womensfootie_id Tinggal tmbah pemain diaspora...              6
4  Timnas Indonesia Vs Timnas Filipina 2-0 Indone...              4


C:\Users\TEMP\AppData\Local\Temp\ipykernel_6692\3514048034.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['topic_cluster'] = np.argmax(topic_distribution, axis=1)


## 4. Interpretasi

In [ ]:
data = data[['full_text', 'topic_cluster']]

data = data.sort_values(by='topic_cluster').reset_index(drop=True)

: 

In [ ]:
data.to_csv('tweet+topics.csv', index=False)

: 

In [ ]:
data

: 

Topik 1: Topik pertama mencakup diskusi tentang sejarah yang dicetak oleh Timnas Indonesia dan pertandingan melawan Filipina di Piala Dunia.

Topik 2: Topik kedua membahas mengenai Indonesia di Piala Dunia, pertandingan melawan Filipina, kualifikasi, dan kelolosan.

Topik 3: Topik ketiga fokus pada perasaan tentang permainan Timnas saat ini, bagaimana mereka bermain, dan harapan terhadap performa mereka.

Topik 4: Topik keempat mencakup saran untuk menambahkan pemain diaspora, diskusi tentang pemain dan strategi yang digunakan.

Topik 5: Topik kelima membahas mengenai fans yang merasa fomo (fear of missing out) terkait dengan Timnas, kemenangan, dan performa mereka.

Topik 6: Topik keenam menyoroti diskusi tentang pelatih Timnas dan strategi yang harus digunakan.

Topik 7: Topik ketujuh mencakup topik tentang pertandingan antara Timnas Indonesia dan Filipina serta kemenangan yang diraih.

Topik 8: Topik kedelapan membahas tentang kemarahan supporter ke Anang dan asyanti pada saat walkout.

Topik 9: Topik kesembilan fokus pada pertandingan U-19 Timnas Indonesia, perjalanan mereka ke Piala Asia U-19, dan performa mereka.

Topik 10: Topik kesepuluh mencakup kemenangan Timnas U-23 Indonesia dan performa impresif mereka di pertandingan.